In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Model, Input, layers
from math import sqrt
from tensorflow import keras
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

from spektral.layers import GCNConv

import keras_tuner as kt
from BaselinePrerocess import gcn_process
from graph import graph_topology
from baselines.gcn import gcn

2023-07-22 00:29:52.020080: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-22 00:29:53.674412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

### Parameters

In [3]:
# ====== preprocessing parameters ======
n_hours = 72
K = 24 
masked_value = 1e-10
split_1 = 0.8
split_2 = 0.9
sigma2 = 0.1
epsilon = 0.5


# ===== model parameters ======
gcn_unit1 = 32
gcn_unit2 = 16
lstm_units = 32
dropout = 0
learning_rate = 5e-4
decay_steps = 10000
decay_rate = 0.95
PATIENCE = 500
EPOCHS = 3000
BATCH = 512

### Preprocess

In [4]:
train_X_mask, val_X_mask, test_X_mask, \
train_ws_y, val_ws_y, test_ws_y, \
scaler, ws_scaler = gcn_process(n_hours, K, masked_value, split_1, split_2)

Index(['WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A', 'TWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B', 'TWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2', 'HWS_S26',
       'TWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
      dtype='object')


In [5]:
print(train_X_mask.shape, val_X_mask.shape, test_X_mask.shape, train_ws_y.shape, val_ws_y.shape, test_ws_y.shape)

(77069, 14, 96) (9634, 14, 96) (19268, 14, 96) (77069, 96) (9634, 96) (19268, 96)


In [6]:
train_X_mask = train_X_mask[:, :, :n_hours]
val_X_mask = val_X_mask[:, :, :n_hours]
test_X_mask = test_X_mask[:, :, :n_hours]

In [7]:
print(train_X_mask.shape, val_X_mask.shape, test_X_mask.shape, train_ws_y.shape, val_ws_y.shape, test_ws_y.shape)

(77069, 14, 72) (9634, 14, 72) (19268, 14, 72) (77069, 96) (9634, 96) (19268, 96)


### Graph topology

In [8]:
train_adj_mat, val_adj_mat, test_adj_mat = graph_topology(n_hours, K, sigma2, epsilon, len(train_ws_y), len(val_ws_y), len(test_ws_y))

node_indices: [ 0  0  0  0  0  1  1  1  2  2  2  3  3  4  4  4  4  5  5  5  6  6  6  7
  7  7  8  8  8  8  8  9  9  9 10 10 10 11 11 11 12 12 12 12 13 13 13 13
 13 13 13 13 13 13 13 13 13 13] 
neighbor_indices: [ 1  4  8 12 13  1  4 13  3  4 13  2 13  0  1  2 13  7  8 13  7  8 13  5
  6 13  0  5  6 12 13 11 12 13 11 12 13  9 10 13  0  9 10 13  0  1  2  3
  4  5  6  7  8  9 10 11 12 13]
number of nodes: 14, number of edges: 58


In [9]:
print(train_adj_mat.shape, val_adj_mat.shape, test_adj_mat.shape)

(77069, 14, 14) (9634, 14, 14) (19268, 14, 14)


### Model

In [11]:
n_nodes = train_adj_mat.shape[-1]
n_timesteps = train_X_mask.shape[-1]

In [12]:
# n_nodes, n_timesteps, gcn1, gcn2, lstm_unit
model, GCNConv = gcn(gcn1=gcn_unit1,
                     gcn2=gcn_unit2,
                     n_nodes=n_nodes,
                     n_timesteps=n_timesteps,
                     lstm_unit=lstm_units,
                     dropout=dropout,
                     masked_value=masked_value
                    )
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 14, 72)]             0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 14, 14)]             0         []                            
                                                                                                  
 gcn_conv_2 (GCNConv)        (None, 14, 32)               2336      ['input_4[0][0]',             
                                                                     'input_3[0][0]']             
                                                                                                  
 gcn_conv_3 (GCNConv)        (None, 14, 16)               528       ['gcn_conv_2[0][0]',    

In [13]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate, 
                                                          decay_steps=decay_steps,
                                                          decay_rate=decay_rate)

model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mse',
              metrics=['mae']
             )


es = EarlyStopping(monitor='val_mae', mode='min', verbose=2, patience=PATIENCE)
mc = ModelCheckpoint('saved_models/gcn_gate_pump_nofuture.h5', 
                     monitor='val_mae', 
                     mode='min', 
                     verbose=2, 
                     custom_objects={'GCNConv': GCNConv},
                     save_best_only=True
                    )


model.fit([train_X_mask, train_adj_mat], train_ws_y,
          validation_data=([test_X_mask, test_adj_mat], test_ws_y),
          epochs=EPOCHS,
          batch_size=BATCH,
          verbose=2,
          shuffle=True,
          callbacks=[es, mc]
         )

Epoch 1/3000


2023-07-22 00:32:16.710568: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0d9401b030 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-22 00:32:16.710623: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2023-07-22 00:32:16.720720: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-22 00:32:16.951844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-07-22 00:32:17.368660: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Epoch 1: val_mae improved from inf to 0.10513, saving model to saved_models/gcn_gate_pump_nofuture.h5
151/151 - 10s - loss: 0.1354 - mae: 0.1862 - val_loss: 0.0188 - val_mae: 0.1051 - 10s/epoch - 64ms/step
Epoch 2/3000


/aul/homes/jshi008/miniconda3/envs/tf213/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_mae improved from 0.10513 to 0.07130, saving model to saved_models/gcn_gate_pump_nofuture.h5
151/151 - 4s - loss: 0.0076 - mae: 0.0637 - val_loss: 0.0084 - val_mae: 0.0713 - 4s/epoch - 24ms/step
Epoch 3/3000

Epoch 3: val_mae improved from 0.07130 to 0.05977, saving model to saved_models/gcn_gate_pump_nofuture.h5
151/151 - 4s - loss: 0.0036 - mae: 0.0453 - val_loss: 0.0058 - val_mae: 0.0598 - 4s/epoch - 25ms/step
Epoch 4/3000

Epoch 4: val_mae improved from 0.05977 to 0.05532, saving model to saved_models/gcn_gate_pump_nofuture.h5
151/151 - 4s - loss: 0.0025 - mae: 0.0380 - val_loss: 0.0049 - val_mae: 0.0553 - 4s/epoch - 25ms/step
Epoch 5/3000

Epoch 5: val_mae improved from 0.05532 to 0.05124, saving model to saved_models/gcn_gate_pump_nofuture.h5
151/151 - 4s - loss: 0.0021 - mae: 0.0345 - val_loss: 0.0042 - val_mae: 0.0512 - 4s/epoch - 25ms/step
Epoch 6/3000

Epoch 6: val_mae improved from 0.05124 to 0.04772, saving model to saved_models/gcn_gate_pump_nofuture.h5
151/1

...

151/151 - 3s - loss: 1.1778e-04 - mae: 0.0066 - val_loss: 4.0546e-04 - val_mae: 0.0147 - 3s/epoch - 22ms/step
Epoch 1012/3000

Epoch 1012: val_mae did not improve from 0.01432
151/151 - 3s - loss: 1.1810e-04 - mae: 0.0066 - val_loss: 4.1357e-04 - val_mae: 0.0149 - 3s/epoch - 22ms/step
Epoch 1013/3000

Epoch 1013: val_mae did not improve from 0.01432
151/151 - 3s - loss: 1.1783e-04 - mae: 0.0066 - val_loss: 4.2478e-04 - val_mae: 0.0150 - 3s/epoch - 22ms/step
Epoch 1014/3000

Epoch 1014: val_mae did not improve from 0.01432
151/151 - 3s - loss: 1.1795e-04 - mae: 0.0066 - val_loss: 4.1182e-04 - val_mae: 0.0148 - 3s/epoch - 22ms/step
Epoch 1015/3000

Epoch 1015: val_mae did not improve from 0.01432
151/151 - 3s - loss: 1.1787e-04 - mae: 0.0066 - val_loss: 3.9580e-04 - val_mae: 0.0145 - 3s/epoch - 23ms/step
Epoch 1016/3000

Epoch 1016: val_mae did not improve from 0.01432
151/151 - 3s - loss: 1.1799e-04 - mae: 0.0066 - val_loss: 4.1437e-04 - val_mae: 0.0149 - 3s/epoch - 22ms/step
Epoch 1017

Epoch 1058/3000

Epoch 1058: val_mae did not improve from 0.01432
151/151 - 3s - loss: 1.1739e-04 - mae: 0.0066 - val_loss: 4.0176e-04 - val_mae: 0.0147 - 3s/epoch - 23ms/step
Epoch 1059/3000

Epoch 1059: val_mae did not improve from 0.01432
151/151 - 3s - loss: 1.1723e-04 - mae: 0.0066 - val_loss: 4.1149e-04 - val_mae: 0.0149 - 3s/epoch - 22ms/step
Epoch 1060/3000

Epoch 1060: val_mae did not improve from 0.01432
151/151 - 3s - loss: 1.1739e-04 - mae: 0.0066 - val_loss: 3.9260e-04 - val_mae: 0.0145 - 3s/epoch - 22ms/step
Epoch 1061/3000

Epoch 1061: val_mae did not improve from 0.01432
151/151 - 3s - loss: 1.1737e-04 - mae: 0.0066 - val_loss: 3.8833e-04 - val_mae: 0.0145 - 3s/epoch - 22ms/step
Epoch 1062/3000

Epoch 1062: val_mae did not improve from 0.01432
151/151 - 3s - loss: 1.1746e-04 - mae: 0.0066 - val_loss: 3.8496e-04 - val_mae: 0.0143 - 3s/epoch - 23ms/step
Epoch 1063/3000

Epoch 1063: val_mae did not improve from 0.01432
151/151 - 3s - loss: 1.1722e-04 - mae: 0.0066 - val_lo

151/151 - 3s - loss: 1.1689e-04 - mae: 0.0066 - val_loss: 4.2337e-04 - val_mae: 0.0150 - 3s/epoch - 22ms/step
Epoch 1105/3000

Epoch 1105: val_mae did not improve from 0.01431
151/151 - 3s - loss: 1.1675e-04 - mae: 0.0066 - val_loss: 4.2194e-04 - val_mae: 0.0150 - 3s/epoch - 23ms/step
Epoch 1106/3000

Epoch 1106: val_mae did not improve from 0.01431
151/151 - 3s - loss: 1.1656e-04 - mae: 0.0065 - val_loss: 4.1970e-04 - val_mae: 0.0150 - 3s/epoch - 22ms/step
Epoch 1107/3000

Epoch 1107: val_mae did not improve from 0.01431
151/151 - 3s - loss: 1.1699e-04 - mae: 0.0066 - val_loss: 4.2173e-04 - val_mae: 0.0150 - 3s/epoch - 22ms/step
Epoch 1108/3000

Epoch 1108: val_mae did not improve from 0.01431
151/151 - 3s - loss: 1.1681e-04 - mae: 0.0066 - val_loss: 4.1501e-04 - val_mae: 0.0149 - 3s/epoch - 22ms/step
Epoch 1109/3000

Epoch 1109: val_mae did not improve from 0.01431
151/151 - 3s - loss: 1.1677e-04 - mae: 0.0066 - val_loss: 4.0145e-04 - val_mae: 0.0147 - 3s/epoch - 22ms/step
Epoch 1110

Epoch 1150/3000


KeyboardInterrupt: 

### Performance

In [14]:
saved_model = load_model('saved_models/gcn_gate_pump_nofuture.h5', custom_objects={'GCNConv': GCNConv})

yhat = saved_model.predict([test_X_mask, test_adj_mat])

inv_yhat = ws_scaler.inverse_transform(yhat)
inv_y = ws_scaler.inverse_transform(test_ws_y)

print("inv_y.shape, inv_yhat.shape", inv_y.shape, inv_yhat.shape)
print('MAE = {}'.format(float("{:.4f}".format(mae(inv_y, inv_yhat)))))
print('RMSE = {}'.format(float("{:.4f}".format(sqrt(mse(inv_y, inv_yhat))))))

603/603 [==============================] - 3s 4ms/step
inv_y.shape, inv_yhat.shape (19268, 96) (19268, 96)
MAE = 0.0747
RMSE = 0.102


In [15]:
wss = ['WS_S1', 'TWS_S25A', 'TWS_S25B', 'TWS_S26']
inv_yhat_reshape = inv_yhat.reshape((-1, 24, 4))
inv_y_reshape = inv_y.reshape((-1, 24, 4))

for i in range(len(wss)):
    plt.rcParams["figure.figsize"] = (15, 3)
    error = inv_yhat_reshape - inv_y_reshape
    plt.plot(error[:, -1, i], linewidth=1)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('Time steps', fontsize=18)
    plt.ylabel('Actual Error (ft)', fontsize=18)
    plt.title('{}'.format(wss[i]), fontsize=18)
    plt.text(9000, -0.6, 'MAE = {}'.format(float("{:.3f}".format(sum(abs(error[:, -1, i]))/len(error)))), fontsize=12)
    plt.show()

In [16]:
up_thre = 0.5
low_thre = -0.5

for i in range(len(wss)):
    plt.rcParams["figure.figsize"] = (15, 3)
    error = inv_yhat_reshape - inv_y_reshape
    error_24h = error[:, -1, i]
    print(error_24h.shape)
    print(np.sum(error[:, -1, i] > up_thre) + np.sum(error[:, -1, i] < low_thre))

(19268,)
11
(19268,)
63
(19268,)
34
(19268,)
34


In [17]:
for i in range(len(wss)):
    plt.rcParams["figure.figsize"] = (15, 3)
    error = inv_yhat_reshape - inv_y_reshape
    error_24h = error[:, -1, i]
    print(error_24h.shape)
    print(np.sum(error[:, -1, i] > up_thre))

(19268,)
5
(19268,)
37
(19268,)
6
(19268,)
28


In [18]:
for i in range(len(wss)):
    plt.rcParams["figure.figsize"] = (15, 3)
    error = inv_yhat_reshape - inv_y_reshape
    error_24h = error[:, -1, i]
    print(error_24h.shape)
    print(np.sum(error[:, -1, i] < low_thre))

(19268,)
6
(19268,)
26
(19268,)
28
(19268,)
6
